## Retrieve MTBS by state from Source coop to test Dask and Docker
For more information: https://source.coop/repositories/cboettig/fire/description

In [1]:
from utils.mtbs_utils import initialize_gee
from utils.source_coop_utils import initialize_dask_cluster,get_usgs_data,get_mtbs_shp
import os
import boto3
from dotenv import load_dotenv
import dask_geopandas as dg

In [ ]:
initialize_gee()

In [ ]:
# Example eusage
client = initialize_dask_cluster()

In [ ]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [ ]:
AWS_ACCESS_KEY_ID

In [ ]:
s3_client = boto3.client('s3',
                         aws_access_key_id = AWS_ACCESS_KEY_ID, 
                         aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                         endpoint_url='https://data.source.coop'
                        )


In [ ]:
file_name= "usgs-mtbs"
usgs_ddf = get_usgs_data(file_name, s3_client)

In [ ]:
file_name="mtbs_perims_DD.shp"
mtbs_shp_ddf=get_mtbs_shp(file_name, s3_client)

In [ ]:
mtbs_shp_ddf

In [ ]:


# # Load the Dask GeoDataFrame
# file_path = "./data/mtbs_perims_DD.shp"
# mtbs_shp_ddf = dg.read_file(file_path, npartitions=4)

# Extract state abbreviation from the 'Event_ID' (first two characters)
mtbs_shp_ddf['State'] = mtbs_shp_ddf['Event_ID'].str[:2]

# Group by 'State' and count the number of fires
fires_by_state = mtbs_shp_ddf.groupby('State').size().compute()

# Convert to a DataFrame for better display
fires_by_state_df = fires_by_state.reset_index()
fires_by_state_df.columns = ['State', 'Fire Count']

# Print the result
print(fires_by_state_df)

In [ ]:
import matplotlib.pyplot as plt
# Sort the DataFrame by Fire Count in descending order
fires_by_state_df = fires_by_state_df.sort_values(by='Fire Count', ascending=False)

# Plot the data as a bar chart
plt.figure(figsize=(15, 8))
plt.bar(fires_by_state_df['State'], fires_by_state_df['Fire Count'], color='orange')

# Add titles and labels
plt.title('Number of Fires by State', fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Fire Count', fontsize=14)

# Add value labels on top of the bars
for index, value in enumerate(fires_by_state_df['Fire Count']):
    plt.text(index, value + 5, str(value), ha='center', va='bottom', fontsize=10)

# Show the plot
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [1]:
import dask_geopandas as dg
import geopandas as gpd
import leafmap
import pandas as pd

# Load the Dask GeoDataFrame
file_path = "./data/mtbs_perims_DD.shp"
mtbs_shp_ddf = dg.read_file(file_path, npartitions=4)

# Filter the data for Wildfire incidents
wildfire_ddf = mtbs_shp_ddf[mtbs_shp_ddf['Incid_Type'] == 'Wildfire']

# Extract state abbreviation from the 'Event_ID' (first two characters)
wildfire_ddf['State'] = wildfire_ddf['Event_ID'].str[:2]

# Group by 'State' and count the number of wildfires
wildfires_by_state = wildfire_ddf.groupby('State').size().compute()

# Convert to a DataFrame for merging
wildfires_by_state_df = wildfires_by_state.reset_index()
wildfires_by_state_df.columns = ['State', 'Wildfire Count']

# Categorize severity levels
def classify_severity(count):
    if count < 50:
        return 'Low'
    elif 50 <= count < 200:
        return 'Medium'
    else:
        return 'High'

wildfires_by_state_df['Severity'] = wildfires_by_state_df['Wildfire Count'].apply(classify_severity)

# Load US states shapefile from Natural Earth
us_states = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_20m.zip")

# Create a column for state abbreviations in the shapefile
state_abbrev_mapping = {
    'Alaska': 'AK', 'Hawaii': 'HI', 'Alabama': 'AL', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL',
    'Georgia': 'GA', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
us_states['State'] = us_states['NAME'].map(state_abbrev_mapping)

# Merge the wildfire counts with the US states shapefile
us_states = us_states.merge(wildfires_by_state_df, on='State', how='left')
us_states['Wildfire Count'] = us_states['Wildfire Count'].fillna(0)
us_states['Severity'] = us_states['Severity'].fillna('Low')


# Define a style callback function for dynamic styling based on severity
def style_callback(feature):
    severity = feature['properties']['Severity']
    color_map = {
        'Low': '#ffffb2',
        'Medium': '#fd8d3c',
        'High': '#bd0026'
    }
    return {
        "fillColor": color_map.get(severity, '#ffffb2'),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7
    }
# Create a map using leafmap
m = leafmap.Map(center=[39.8283, -98.5795], zoom=4)
# Add the US states layer with wildfire counts
m.add_gdf(
    us_states,
    layer_name='Wildfires by State',
    info_mode='on_click',
    style_callback=style_callback,
)

# Add a legend
m.add_legend(
    title="Wildfire Severity by State",
    labels=['Low', 'Medium', 'High'],
    colors=['#ffffb2', '#fd8d3c', '#bd0026']
)


In [2]:
m

Map(center=[39.8283, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
m = leafmap.Map(center=[0, 0], zoom=2)
url = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson"
m.add_geojson(
    url, layer_name="Countries", fill_colors=["red", "yellow", "green", "orange"]
)
m

In [9]:
raster= "./wildfire_severity.tif"

In [10]:
m.add_raster(raster, colormap="terrain", layer_name="raster")

In [ ]:
# Display the map
m

In [ ]:
mtbs_shp_ddf.compute()

In [ ]:
ddf=usgs_ddf.compute()

In [ ]:
ddf.column()

In [ ]:
mtbs_ddf.head()

In [ ]:
print(mtbs_ddf.columns)

In [ ]:
mtbs_ddf.columns = mtbs_ddf.columns.str.strip()

In [ ]:
print(mtbs_ddf['Ig_Date'].dtype)

In [ ]:
mtbs_ddf['Ig_Date'] = dd.to_datetime(mtbs_ddf['Ig_Date'], errors='coerce')

In [ ]:
grouped = mtbs_ddf.groupby('Ig_Date')
print(grouped)

In [ ]:
ddf.plot()